In [7]:
import os
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.models as models
from torch.utils.data import DataLoader, Dataset
from PIL import Image
from sklearn.metrics import accuracy_score
import pickle

# Define dataset class
class CustomDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = []
        self.labels = []
        
        # Loop through the directories (YES and NO)
        for label in ['YES', 'NO']:
            label_path = os.path.join(root_dir, label)
            for image_name in os.listdir(label_path):
                if image_name.endswith(('.jpg', '.png', '.jpeg')):  # Filter image files
                    self.image_paths.append(os.path.join(label_path, image_name))
                    self.labels.append(0 if label == 'NO' else 1)  # 0 for NO, 1 for YES

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert("RGB")  # Convert to RGB

        if self.transform:
            image = self.transform(image)

        label = self.labels[idx]
        return image, label

# Define transformations for the images
transform = transforms.Compose([
    transforms.Resize((224, 224)),   # Resize image to 224x224 (MobileNet input size)
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # MobileNet standard normalization
])

# Load datasets
train_dataset = CustomDataset(root_dir=r"D:\Prayag Files\TIET\Extras\hackathons\analog\TRAIN", transform=transform)
val_dataset = CustomDataset(root_dir=r"D:\Prayag Files\TIET\Extras\hackathons\analog\VAL", transform=transform)
test_dataset = CustomDataset(root_dir=r"D:\Prayag Files\TIET\Extras\hackathons\analog\TEST", transform=transform)

# DataLoader
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Define the MobileNetV2 model
model = models.mobilenet_v2(pretrained=True)
model.classifier[1] = nn.Linear(model.classifier[1].in_features, 2)  # 2 classes: YES, NO

# Move model to device (GPU or CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training the model
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}, Accuracy: {100 * correct / total:.2f}%")

# Saving the model as a .pkl file
model_path = r"D:\Prayag Files\TIET\Extras\hackathons\analog\mobilenet_model.pkl"
with open(model_path, 'wb') as f:
    pickle.dump(model, f)

print(f"Model saved to {model_path}")

# Evaluating the model on the validation set
model.eval()
val_correct = 0
val_total = 0
val_labels_list = []
val_preds_list = []

with torch.no_grad():
    for images, labels in val_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        val_total += labels.size(0)
        val_correct += (predicted == labels).sum().item()
        val_labels_list.extend(labels.cpu().numpy())
        val_preds_list.extend(predicted.cpu().numpy())

val_accuracy = 100 * val_correct / val_total
print(f"Validation Accuracy: {val_accuracy:.2f}%")

# Testing the model on the test set
test_correct = 0
test_total = 0
test_labels_list = []
test_preds_list = []

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        test_total += labels.size(0)
        test_correct += (predicted == labels).sum().item()
        test_labels_list.extend(labels.cpu().numpy())
        test_preds_list.extend(predicted.cpu().numpy())

test_accuracy = 100 * test_correct / test_total
print(f"Test Accuracy: {test_accuracy:.2f}%")

# Print classification report (optional)
from sklearn.metrics import classification_report
print("\nClassification Report:")
print(classification_report(test_labels_list, test_preds_list))


c:\Users\Prayag Chawla\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Prayag Chawla\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to C:\Users\Prayag Chawla/.cache\torch\hub\checkpoints\mobilenet_v2-b0353104.pth
100%|██████████| 13.6M/13.6M [00:05<00:00, 2.67MB/s]


Epoch [1/10], Loss: 0.2124, Accuracy: 90.94%
Epoch [2/10], Loss: 0.0472, Accuracy: 98.72%
Epoch [3/10], Loss: 0.0660, Accuracy: 97.72%
Epoch [4/10], Loss: 0.0443, Accuracy: 98.56%
Epoch [5/10], Loss: 0.0418, Accuracy: 98.56%
Epoch [6/10], Loss: 0.0247, Accuracy: 99.17%
Epoch [7/10], Loss: 0.0084, Accuracy: 99.72%
Epoch [8/10], Loss: 0.0106, Accuracy: 99.72%
Epoch [9/10], Loss: 0.0147, Accuracy: 99.50%
Epoch [10/10], Loss: 0.0452, Accuracy: 98.39%
Model saved to D:\Prayag Files\TIET\Extras\hackathons\analog\mobilenet_model.pkl
Validation Accuracy: 96.17%
Test Accuracy: 96.50%

Classification Report:
              precision    recall  f1-score   support

           0       0.94      1.00      0.97       300
           1       1.00      0.93      0.96       300

    accuracy                           0.96       600
   macro avg       0.97      0.97      0.96       600
weighted avg       0.97      0.96      0.96       600



In [2]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Load the base MobileNetV2 model without the top layer for fine-tuning
base_model = tf.keras.applications.MobileNetV2(input_shape=(224, 224, 3),
                                               weights='imagenet', include_top=False)

# Freeze the base model layers for transfer learning
base_model.trainable = False

# Start defining the new model using the functional API
inputs = layers.Input(shape=(224, 224, 3))
x = base_model(inputs)
x = layers.GlobalAveragePooling2D()(x)

# Add the final Dense layer for classification
final_dense = layers.Dense(1, activation='sigmoid', kernel_initializer='he_normal')(x)

# Create the final model
model = models.Model(inputs, final_dense)

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Data Preparation
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

train_dir = r"D:\Prayag Files\TIET\Extras\hackathons\analog\TRAIN"
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
)

val_dir = r"D:\Prayag Files\TIET\Extras\hackathons\analog\VAL"
val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
)

# Train the model
model.fit(train_generator, epochs=10, validation_data=val_generator)

# Evaluate the model on the validation set and print the final accuracy
val_loss, val_accuracy = model.evaluate(val_generator)
print(f"Final Validation Accuracy: {val_accuracy * 100:.2f}%")

# Save the model
model.save(r"D:\Prayag Files\TIET\Extras\hackathons\analog\mobilenet_model.h5")
print("Model saved to D:\\Prayag Files\\TIET\\Extras\\hackathons\\analog\\mobilenet_model.h5")

# Convert the model to TensorFlow Lite format for optimization
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]  # Enable optimizations (quantization)
tflite_model = converter.convert()

# Save the TensorFlow Lite model
tflite_model_path = r"D:\Prayag Files\TIET\Extras\hackathons\analog\mobilenet_model.tflite"
with open(tflite_model_path, 'wb') as f:
    f.write(tflite_model)

print(f"Model saved as TFLite to {tflite_model_path}")


Found 1800 images belonging to 4 classes.
Found 600 images belonging to 4 classes.
Epoch 1/10


c:\Users\Prayag Chawla\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


57/57 ━━━━━━━━━━━━━━━━━━━━ 48s 690ms/step - accuracy: 0.5001 - loss: -6.6900 - val_accuracy: 0.5000 - val_loss: -24.1555
Epoch 2/10
57/57 ━━━━━━━━━━━━━━━━━━━━ 42s 722ms/step - accuracy: 0.4942 - loss: -31.9660 - val_accuracy: 0.5000 - val_loss: -48.4653
Epoch 3/10
57/57 ━━━━━━━━━━━━━━━━━━━━ 36s 621ms/step - accuracy: 0.4989 - loss: -57.6243 - val_accuracy: 0.5000 - val_loss: -73.0649
Epoch 4/10
57/57 ━━━━━━━━━━━━━━━━━━━━ 32s 552ms/step - accuracy: 0.5179 - loss: -80.0580 - val_accuracy: 0.5000 - val_loss: -97.3219
Epoch 5/10
57/57 ━━━━━━━━━━━━━━━━━━━━ 35s 616ms/step - accuracy: 0.5013 - loss: -108.8962 - val_accuracy: 0.5000 - val_loss: -121.9189
Epoch 6/10
57/57 ━━━━━━━━━━━━━━━━━━━━ 36s 611ms/step - accuracy: 0.5055 - loss: -133.7342 - val_accuracy: 0.5000 - val_loss: -146.2515
Epoch 7/10
57/57 ━━━━━━━━━━━━━━━━━━━━ 31s 543ms/step - accuracy: 0.5102 - loss: -158.9789 - val_accuracy: 0.5000 - val_loss: -170.7995
Epoch 8/10
57/57 ━━━━━━━━━━━━━━━━━━━━ 35s 611ms/step - accuracy: 0.5013 - l

Final Validation Accuracy: 50.00%
Model saved to D:\Prayag Files\TIET\Extras\hackathons\analog\mobilenet_model.h5
INFO:tensorflow:Assets written to: C:\Users\Prayag Chawla\AppData\Local\Temp\tmpsqcsid4y\assets


INFO:tensorflow:Assets written to: C:\Users\Prayag Chawla\AppData\Local\Temp\tmpsqcsid4y\assets


: 